# Strategy Backtesting Using Python (DMP-1)
#### Notebook Created on:  25 Jan 2018
##### Last Update: 6 August 2018
##### Author: Vivek Krishnamoorthy

### Taking stock of where we are currently

We covered quite a bit of ground when we met last. I expect that you would have spent time reviewing it. 

One effective way to learn is to mess with the code I share with you. Tweak the values, create new variables and see what happens. Play around with the cells in the Jupyter Notebook to familiarize yourself with the Python syntax and figure out its features and limitations. **The benefits of running and experimenting with the shared code line by line yourself *by typing it in* will be a far better use of your time and effort than just reading about it.**

There're tons of references I'd provided in my previous class in case you want to delve deeper (I hope you have).

#### Some myths about programming ...

1. It's too late for me to learn it.

2. I need to have an engineering or computer science background to become a quant/algo trader.

3. I will have acquired all the needed knowledge if I go through the EPAT programming oriented sessions.

4. I have to be an expert level programmer to code trading strategies.

#### ... and some truisms

1. It's never too late. The internet is filled with inspiring experiences about people who learnt it quite 'late' in life. Check [this](https://www.google.co.in/search?hl=en&source=hp&ei=P_UpW9yqKcv5vgTs_oXYCw&q=programming+learning+late&oq=programming+learning+late&gs_l=psy-ab.3..0i22i30k1l10.1999.6123.0.6329.27.22.0.3.3.0.279.2152.0j13j1.15.0....0...1c.1.64.psy-ab..9.17.2170.0..0j0i131k1.130.ZbcQA9G_094) out.

2. Not true. To get comfortable with something, it has to start coming naturally to you. This will happen only if you practise everyday. I know I've said this ad nauseum. But really, there's no getting around it. Keep at it and you'll find that it gets easier and more fun.

3. It doesn't take very long to learn programming but months to become an expert. Read widely, watch videos, try stuff, fail, try again (Lather, rinse, repeat).

4. Not true. If you understand financial markets and are familiar with Python or R, you're good to go. The coding community is a vibrant and giving one. You'll find a lot of ninja level programmers actively engaged in troubleshooting and helping newcomers on online forums like [Stackoverflow](https://stackoverflow.com/). Use it when you encounter issues.

#### A handy and step-wise approach to Strategy Testing

1. Come up with a trading/strategy idea (this could come from something you read, or may be a gut instinct that needs to be tested - say, buying at Friday close and selling on the following Monday at market open).

2. Download the relevant data. Daily historical data is usually quite straightfoward to obtain, especially for stocks. Higher frequency data (5 minutely, minutely, etc.) however is expensive and you need to check this with your broker.

3. Think about the algorithm (a series of steps to buy and sell)
    * Build the indicator
    * Create the rule
    * Calculate returns, P & L, etc.
    
4. Do it stepwise and test each part once you write it.

5. Make sure you annotate your code with comments. If not, you'll spend hours trying to understand your own code when you revisit it after a few days.

6. Fragment the code. You don't need to run the data download part every single time. Save the data download part as a function or a separate script/file.

7. Contingency plan
    * What if the code to download data automatically doesn't work?
    * Download the data manually as a csv file and use it instead.


```python

# The first step when we start to write a Python program is to import all the necessary
# libraries/packages with aliases if required. The aliases I've used are quite standardized 
# in scientific Python code that you come across

import pandas as pd
import numpy as np
import datetime
pd.core.common.is_list_like = pd.api.types.is_list_like
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import os

os.getcwd()
os.chdir("C:\Work\EPAT\Python\Strategies") # You'll need to key in the directory relevant to your computer
# Here's where all the files pertaining to your program should be stored
```

[pandas-datareader](http://pandas-datareader.readthedocs.io/en/latest/) is a popular library (other libraries include [quandl](https://www.quandl.com/tools/python), [nsepy](http://nsepy.readthedocs.io/en/latest/) etc.) used to pull financial data automatically from different web sources. However, the data sources often change and one needs to check the documentation from time to time to be updated. A few months ago, I would always download whatever data I needed through Yahoo Finance or Google Finance (when using `pandas-datareader`). They don't work anymore.

### A short detour - Dates and Times in Python

In Python ,there are a number of ways to represent dates, times, and time intervals between them. `pandas` and `NumPy` have their own ways of representing them. We will be mostly dealing with the `pandas` date formats since that is most relevant to quantitative finance. Nevertheless, it's useful to know how they relate with each other.

Two important methods within `datetime` which are also the same in `pandas` are `strptime` and `strftime`.
* `strptime` parses a string and **converts it to a `datetime` object**.
* `strftime` **converts a `datetime` object to a specified format string**.

As a mnemonic, I find it helpful to link the `p` in `strptime` to
* *parsing* a string to get a datetime object. 

And the `f` in `strftime` to 
* *formatting*  a `datetime` object to get a formatted string of our choice.

*I'd strongly suggest that you go through the use cases for these two methods and try them out (even if you don't try out the rest of the code pertaining to dates and times). Dealing with different formats of dates and times from disparate source files (since data could come from many potential data providers) is a pain we often encounter while dealing with financial data. [Here's](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) the official documentation on them.*

Native Python has a built-in `datetime` library and a third-party library [`dateutil`](https://dateutil.readthedocs.io/en/stable/) which supplies `date`, `datetime`, `time`, `timedelta` and other objects. There are many useful functionalities that you can perform using them. However, while working with large datasets, they are inefficient compared to better available date formats in `pandas` or `NumPy`.

`NumPy` has the `datetime64` data type which encodes dates as 64-bit integers. It's named so because the name `datetime` was already taken by the `datetime` library in native Python. Some of the objects follow the convenient nomenclature of `64` suffixed at the end of their native Python names. Ex. `datetime64`, `timedelta64`.

`pandas` provides a `Timestamp` object which is like the best of both worlds. It's got the user-friendliness of `datetime` and `dateutil` and the efficiency of `NumPy's datetime64`. It is interchangeable with Python's `datetime` in most cases.

**`DatetimeIndex` is a `pandas` index object formed by combining multiple entries of `Timestamp` objects.**

***Some use cases which you can try on your own time if you wish.***

```python
## Native Python
import datetime
a = datetime.datetime(year=2018, month=6, day=10)
print("a", type(a))

a2 = datetime.datetime.now()
print("a2", type(a2))

a3 = datetime.datetime.now().date()
print("a3", type(a3))

a3.strftime("%A")
a2.strftime("%a")

a4date = "18 Nov 2011"
a5 = datetime.datetime.strptime(a4date,"%d %b %Y").date()

from dateutil import parser
a5 = parser.parse("10th Oct 2014")
a5
print("a5", type(a5))
```

```python
## NumPy Datetimes

import numpy as np
b = np.datetime64("2016-04-15")

b2 = np.datetime64("2017-11") # only specifying month and year

b3 = np.arange('2015-08', '2015-09', dtype='datetime64[D]') # getting all the dates for a month

np.datetime64('2018-01-01') + np.timedelta64(25, 'D')
```

```python
## pandas Timestamps

import pandas as pd

c = pd.to_datetime("10th Sept 2016")
c
print("c", type(c))
c.strftime("%B")

c2 = c + pd.to_timedelta(np.arange(20), 'D')
c2
print("c2", type(c2))

c3 = pd.Timestamp('2017-03-15')
c3
print("c3", type(c3))

```

You can read and try some examples on this from [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html).

### Downloading and loading the data in a pandas DataFrame

We will now use the `pandas-datareader` to automatically download the required data for the Microsoft stock price. In case you're having trouble with the download, please use the `csv` file that's been provided to you and run the relevant code.


```python

end = datetime.datetime.now().date()
start = end - pd.Timedelta(days = 365 * 10)
df = web.DataReader("MSFT", "quandl", start, end)

## IMPORTANT: Please run the below line only if your pandas-datareader doesn't work

df = pd.read_csv("MSFT.csv", index_col=0, parse_dates=True)

```

Let's now examine `df`, our `pandas DataFrame` closely.

```python

df.head()
df.tail()
df.shape
df.describe()
df.info()
```

You'll notice that the order of entries is from the latest to the oldest. This is the way in which data gets downloaded via Quandl. We will encounter quirks such as these with different data vendors. And a significant part of our job when we work on programming quant trading strategies, is data munging and cleaning.

```python
# We reindex our dataframe so as to get the data ordered the way we'd prefer

df = df.reindex(index=df.index[::-1])
df.head()
df.tail()

```

### Visualizing your Data

Let me remind you that [Matplotlib](https://matplotlib.org/) is a plotting library extensively used in quantitative finance and other fields to produce pretty graphs and figures. We'll be primarily using it as a visualization tool.

Visualizing is a nice way to get a 'feel' for your dataset and understand the story that your data tells you.

We'll first take a look at the `matplotlib` library's `plot()` function.

You can think of it as a quick and dirty way (this method would be like comfort food for MATLAB users who are used to this kind of API) to get a visual representation of your data. 

The below method uses a MATLAB-style API. It's easy and convenient but it comes at a cost. The simpler customizations that you may want such as manipulating the axes, adding labels, etc. can be easily done. 

However, if you want to say, change the font used, plot multiple charts in one figure, use LaTeX for scientific notations, etc., you'll need to use the other method (let's call it the Python-style API for convenience).
```python

plt.figure(figsize=(14, 9))
plt.plot(df['AdjHigh'], linewidth=0.5, label='Adjusted High')
plt.plot(df['AdjLow'], linewidth=0.5, label='Adjusted Low')
plt.plot(df['AdjClose'], linewidth=0.5, label='Adjusted Close')
plt.grid(True)
plt.title('Microsoft Price Movements')
plt.xlabel('Time')
plt.ylabel('Price')
#plt.axis('tight')
#plt.tight_layout()
plt.legend()
plt.show()

```

In [ ]:
plt.figure(figsize=(14, 9))
plt.plot(df['AdjHigh'], linewidth=0.5, label='Adjusted High')
plt.plot(df['AdjLow'], linewidth=0.5, label='Adjusted Low')
plt.plot(df['AdjClose'], linewidth=0.5, label='Adjusted Close')
plt.grid(True)
plt.title('Microsoft Price Movements')
plt.xlabel('Time')
plt.ylabel('Price')
#plt.axis('tight')
#plt.tight_layout()
plt.legend()
plt.show()

The Python-syle API exploits the Object Oriented API style of Python. I've provided two illustrative examples below. Notice the difference in styles.
There's a bit more typing to do here. It may not appear obvious why this is a better method in the long run. Suffice to say that as you start producing more complex charts, the more explicit use of objects makes it more customizable to suit your needs.

We will not be using this method for today. However, I'm flagging it as a topic that can be explored as you get better at writing programs. [This](http://www.scipy-lectures.org/intro/matplotlib/index.html) is a good place to rev up your charting skills.

```python
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 9))
### subplots() creates axes on a chart
for i in ['AdjHigh', 'AdjLow', 'AdjClose']:
    curr_label = i
    ax.plot(df[i], linewidth=0.5, label=curr_label)
ax.legend()
ax.grid()
ax.set
plt.show()


fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(14, 9))
ax1.plot(df['AdjHigh'], linewidth=0.5, label='Adjusted High')
ax1.plot(df['AdjLow'], linewidth=0.5, label='Adjusted Low')
ax1.plot(df['AdjClose'], linewidth=0.5, label='Adjusted Close')
ax1.grid(True)
ax1.legend()
ax2=ax1.twinx() # plots the specified dataset using the second axis (right)
ax2.plot(df['AdjVolume'], linewidth=1, label='Volume')
ax2.legend()
plt.show()

```

An even quicker way to plot graphs when you're dealing with `pandas` is the `.plot()` method directly as shown. The `.plot()` method is a wrapper around `matplotlib's` `plot()` function.

```python

df[['AdjHigh', 'AdjLow', 'AdjClose']].plot(grid=True, linewidth=0.5, figsize=(14, 9 ))
df['AdjVolume'].plot(grid=True, linewidth=0.5, figsize=(14, 9))
```

### Strategy # 1: The Modified Faber Strategy

One of the most downloaded papers on the SSRN portal is [A Quantitative Approach to Tactical Asset Allocation](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=962461) by Meb Faber. We shall refer to this approach as the 'Faber strategy'.

We modify the Faber strategy to demonstrate a somewhat simplified version of the same.

We shall compute the 200-day Simple Moving Average (henceforth referred to as 'SMA200' or '10-month SMA') and compare it with the Close prices each day. There are two conditions which we shall check each day.

1. If the price is greater than the SMA200, we will buy one Microsoft stock. We shall then continue to stay invested until the Close price dips below SMA200.

2. When this happens, we will sell our MSFT stock and stay out of the market until our first condition is again satisfied.

Our trading rules can be stated as

* Buy when Price > 10-month SMA (since there are approx. 200 trading days in 10 months)
* Sell and move to Cash when Price < 10-month SMA





```python
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
```

```python
buyPrice = 0
sellPrice = 0
maWealth = 1.0
cash = 1
stock = 0
sma = 200

ma = np.round(df1['AdjClose'].rolling(window=sma, center=False).mean(), 2)
n_days = len(df1['AdjClose'])

# ma.iloc[sma-2:205]
```

```python

closePrices = df1['AdjClose']

buy_data = []
sell_data = []

trade_price = []

wealth = []

for d in range(sma-1, n_days):
    # Buy if stock price > MA & if not bought yet
    if closePrices[d] > ma[d] and cash == 1:
        buyPrice = closePrices[d + 1]
        buy_data.append(buyPrice)
        trade_price.append(buyPrice)
        cash = 0
        stock = 1
        
    # Sell if stock price < MA and if you have a stock to sell
    
    if closePrices[d] < ma[d] and stock == 1:
        sellPrice = closePrices[d + 1]
        sell_data.append(sellPrice)
        trade_price.append(sellPrice)
        cash = 1
        stock = 0
        
        maWealth = maWealth * (sellPrice / buyPrice)
        wealth.append(maWealth)

tp = pd.DataFrame(trade_price)
wealth.insert(0, 1)
w = pd.DataFrame(wealth)
plt.plot(w)
```

Plotting the 10-month SMA and the Close prices

```python

df1['SMA200'] = ma

plt.figure(figsize=(14, 9))
plt.plot(df1['AdjClose'], linewidth=0.5, label='Adjusted Close')
plt.plot(df1['SMA200'], linewidth=0.5, label='200-day Moving Average')
plt.grid(True)
plt.title('Microsoft Stock Price History')
plt.xlabel('Time')
plt.ylabel('Price')
#plt.axis('tight')
#plt.tight_layout()
plt.legend()
plt.show()
# %matplotlib inline
# %matplotlib qt
```


### Strategy # 2: The Moving Average Crossover Strategy

This is a well-known trading technique and considered to be the 'Hello World' of algorithmic/quantitative trading. It's essentially a way to take advantage of the momentum or trend in stock price movements.

We shall have two SMA filters viz. the shorter lookback period SMA (henceforth referred to as  'SMA50') and the longer lookback period SMA (henceforth referred to as 'SMA150'). We will go long on Microsoft *at the first instance* when the SMA50 exceeds the SMA150. Similarly, we will go short on Microsoft, *at the first instance* when the SMA150 exceeds the SMA50. This strategy can work well when the price movements of the asset trend in a particular direction (upwards or downwards) persistently before slowly reversing their drift.

Our trading rules can be stated as

* Go long when SMA50 > SMA150 on a given day and SMA50 < SMA150 on the previous day
* Go short when SMA50 < SMA150 on a given day and SMA50 > SMA150 on the previous day



```python

m = 50 # defining the shorter lookback period
n = 150 # defining the longer lookback period

df2['SMA50'] = df2['AdjClose'].rolling(window=m, center=False).mean()
df2['SMA150'] = df2['AdjClose'].rolling(window=n, center=False).mean()
```

In [ ]:
df2 = df.copy()

We'll now plot the key variables and see how they look.
```python

df2[['AdjClose', 'SMA50', 'SMA150']].plot(grid=True, linewidth=0.5)
```

In [ ]:
df2.head()

In [ ]:
df2.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'ExDividend'], axis=1, inplace=True)

```python

df2['SMA50_2'] = df2['AdjClose'].rolling(window=m, center=False).mean().shift(1)
df2['SMA150_2'] = df2['AdjClose'].rolling(window=n, center=False).mean().shift(1)
```

We've now computed all of what we need to generate trading signals. Let's go ahead and now do that.


In [ ]:
m = 50 # defining the shorter lookback period
n = 150 # defining the longer lookback period

df2['SMA50'] = df2['AdjClose'].rolling(window=m, center=False).mean()
df2['SMA150'] = df2['AdjClose'].rolling(window=n, center=False).mean()

df2['SMA50_2'] = df2['AdjClose'].rolling(window=m, center=False).mean().shift(1)
df2['SMA150_2'] = df2['AdjClose'].rolling(window=n, center=False).mean().shift(1)

In [ ]:
df2['Signal'] = np.where((df2['SMA50'] > df2['SMA150']) 
                        & (df2['SMA50_2'] < df2['SMA150_2']), 1, 0)
df2['Signal'] = np.where((df2['SMA50'] < df2['SMA150']) 
                        & (df2['SMA50_2'] > df2['SMA150_2']), -1, df2['Signal'])


In [ ]:
df2.head()

```python
df2['Signal'] = np.where((df2['SMA50'] > df2['SMA150']) 
                        & (df2['SMA50_2'] < df2['SMA150_2']), 1, 0)
df2['Signal'] = np.where((df2['SMA50'] < df2['SMA150']) 
                        & (df2['SMA50_2'] > df2['SMA150_2']), -1, df2['Signal'])
```

In [ ]:
df2['Buy'] = df2.apply(lambda x : x['AdjClose'] if x['SMA50'] > x['SMA150'] 
                        and x['SMA50_2'] < x['SMA150_2'] else 0, axis=1)

df2['Sell'] = df2.apply(lambda y : -y['AdjClose'] if y['SMA50'] < y['SMA150'] 
                        and y['SMA50_2'] > y['SMA150_2'] else 0, axis=1)

df2['TP'] = df2['Buy'] + df2['Sell']
df2['TP']=df2['TP'].replace(to_replace=0, method='ffill')

df2['Position'] = df2['Signal'].replace(to_replace=0, method= 'ffill')
k = df2['TP'].nonzero()

type(k) # this tells us k is a tuple

k[0] #this gives us a numpy array
type(k[0]) #confirms that this is a numpy ndarray
len(k[0]) # total number of positions

frame = df2.iloc[k]

In [ ]:
frame.head()

In [ ]:
df2['Signal'].value_counts()
# In the period, we've chosen, this is in line with what we see in k

In [ ]:
# Alternate way to plot the graph
plt.figure(figsize=(10, 5))
plt.plot(df2['Position'])
plt.title("Signal showing buying/selling positions")
plt.xlabel('Time')
plt.tight_layout()
plt.show()

In [ ]:
df2['Buy & Hold Returns'] = np.log(df2['AdjClose'] / df2['AdjClose'].shift(1))
df2['Strategy Returns'] = df2['Buy & Hold Returns'] * df2['Position'].shift(1)

df2[['Buy & Hold Returns', 'Strategy Returns']].cumsum().plot(grid=True, figsize=(9,5))


### Downloading Data for Multiple Stocks

We can download data for stocks in multiple ways using the APIs provided by data vendors.
Data vendors provide the the data in different formats and so the methods used to download it, and clean it vary slightly. I've adapted the approach from [here](https://gist.github.com/CNuge).

Please ensure that your `pandas-datareader` is installed and working.

```python

import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
from concurrent import futures
```
The `futures` module in the library `concurrent` provides a high level API to perform parallel tasks. This works like a charm when you have a lot of data to download since it can happen in parallel.



In [ ]:
def download_stocks(stock):
    try:
        print('Trying to retrieve the %s symbol...\n' % (stock))
        stock_df = web.DataReader(stock, 'quandl', start, end)
        stock_df['Name'] = stock
        output_name = stock + '.csv'
        list_stocks.append(output_name)
        stock_df.to_csv(output_name)
        print('Symbol %s downloaded OK.\n' % (stock))
    except:
        bad_tickers.append(stock)
        print('Problems downloading the %s symbol.\n' % (stock))

In [ ]:
stocksList = ['MSFT', 'AAPL', 'IBM', 'DELL', 'HP', 
              'TSLA', 'AMZN', 'AIG', 'STJ', 'WMT', 
              'LVLT', 'SBUX', 'CL', 'NKE', 'FOSL']
list_stocks = []

### Keeping track of failed ticker download queries
bad_tickers = []

end = datetime.datetime.now().date()
start = end - pd.Timedelta(days = 365 * 5)

# We use the concurrent.futures module's ThreadPoolExecutor 
# to speed up the downloads by doing them in parallel rather than serially
    
### Set the maximum thread number
max_workers = 50

workers = min(max_workers, len(stocksList)) 
### In case a smaller number of stocks than threads was passed in
with futures.ThreadPoolExecutor(workers) as executor:
    res = executor.map(download_stocks, stocksList)
### map allows multiple calls to the given function. It passes each of the items in stocksList (any 
### iterable) to the function. Here, it can happen concurrently because of the ThreadPoolExecutor()
### subClass

#### References

1. The `pandas-datareader` library documentation - http://pandas-datareader.readthedocs.io/en/latest/#
2. The Python `datetime` documentation - https://docs.python.org/3/library/datetime.html#module-datetime
3. The `dateutil` library documentation - https://dateutil.readthedocs.io/en/stable/
4. Examples for dealing with dates and times in Python - https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html
3. Faber, Meb, A Quantitative Approach to Tactical Asset Allocation (February 1, 2013). The Journal of Wealth Management, Spring 2007 . Available at SSRN: https://ssrn.com/abstract=962461
3. Documentation on the `.nonzero()` method in `NumPy` : https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.nonzero.html
4. Documentation on `pandas DataFrame .apply()` function: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html
5. https://stackoverflow.com/
6. `MatPlotlib` tutorial - http://www.scipy-lectures.org/intro/matplotlib/index.html
